# Model 3 Random Forests

In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

## Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("Resources/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


## Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
target = df["koi_disposition"]
target_names = ["confirmed", "false positive", "candidate"]

data = df.drop('koi_disposition', axis=1)
data = data.drop('koi_tce_plnt_num', axis=1)
feature_names = data.columns
data.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


## Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [7]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6122,0,0,0,0,6.768901,7.380000e-05,-7.380000e-05,133.077240,0.008440,-0.008440,...,-171,4.327,0.153,-0.187,1.125,0.310,-0.207,294.40472,39.351681,14.725
6370,0,1,0,1,0.733726,6.060000e-06,-6.060000e-06,132.020050,0.007950,-0.007950,...,-175,4.578,0.033,-0.187,0.797,0.211,-0.056,284.50391,42.463860,15.770
2879,1,0,0,0,7.652707,6.540000e-05,-6.540000e-05,134.460380,0.006190,-0.006190,...,-189,4.481,0.050,-0.200,0.963,0.290,-0.097,295.50211,38.983540,13.099
107,0,0,0,0,7.953547,1.910000e-05,-1.910000e-05,174.662240,0.001820,-0.001820,...,-85,4.536,0.056,-0.016,0.779,0.023,-0.049,291.15878,40.750271,15.660
29,0,0,0,0,4.959319,5.150000e-07,-5.150000e-07,172.258529,0.000083,-0.000083,...,-77,4.359,0.110,-0.110,1.082,0.173,-0.130,292.16705,48.727589,15.263


## Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
# Scale your data
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Train the Model



In [9]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, encoded_y_train)

In [10]:
print(f"Training Data Score: {rf.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {rf.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.9010297482837528


In [11]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances

array([0.09987269, 0.0673311 , 0.11251696, 0.03368992, 0.02259924,
       0.01839065, 0.01616273, 0.01317104, 0.02390952, 0.02385875,
       0.01973457, 0.01173882, 0.01094772, 0.02163507, 0.03581116,
       0.03278717, 0.01993106, 0.01496788, 0.01402194, 0.04913188,
       0.03047953, 0.03471357, 0.01499779, 0.012895  , 0.0173954 ,
       0.0121854 , 0.05398741, 0.00962338, 0.03384478, 0.0279948 ,
       0.00853584, 0.00875387, 0.01067067, 0.00843542, 0.01101744,
       0.00846475, 0.01198086, 0.01130531, 0.0105089 ])

In [12]:
# Sort the features by their importance
sorted(zip(rf.feature_importances_, data.columns), reverse=True)

[(0.11251696265705541, 'koi_fpflag_co'),
 (0.09987268950898721, 'koi_fpflag_nt'),
 (0.06733110346042798, 'koi_fpflag_ss'),
 (0.05398740755836139, 'koi_model_snr'),
 (0.049131882387368056, 'koi_prad'),
 (0.035811157880481906, 'koi_duration_err1'),
 (0.03471357475676893, 'koi_prad_err2'),
 (0.03384477621286185, 'koi_steff_err1'),
 (0.0336899191361595, 'koi_fpflag_ec'),
 (0.03278717196354169, 'koi_duration_err2'),
 (0.03047953091617997, 'koi_prad_err1'),
 (0.02799479799546814, 'koi_steff_err2'),
 (0.0239095161022474, 'koi_time0bk_err1'),
 (0.023858746831343383, 'koi_time0bk_err2'),
 (0.02259924400690978, 'koi_period'),
 (0.021635073601588953, 'koi_duration'),
 (0.019931061734551392, 'koi_depth'),
 (0.019734572154728118, 'koi_impact'),
 (0.018390648512301076, 'koi_period_err1'),
 (0.017395401639650553, 'koi_insol_err1'),
 (0.016162725276904676, 'koi_period_err2'),
 (0.014997789622431305, 'koi_teq'),
 (0.014967883497351876, 'koi_depth_err1'),
 (0.014021938304412375, 'koi_depth_err2'),
 (0.0

In [13]:
from sklearn.metrics import classification_report

predictions = rf.predict(X_test_scaled)
print(classification_report(encoded_y_test, predictions, 
                            target_names = ["confirmed", "false positive", "candidate"]))

                precision    recall  f1-score   support

     confirmed       0.83      0.75      0.79       411
false positive       0.83      0.86      0.85       484
     candidate       0.97      1.00      0.98       853

      accuracy                           0.90      1748
     macro avg       0.88      0.87      0.87      1748
  weighted avg       0.90      0.90      0.90      1748



# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [14]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [400], 
              'max_features' : ['auto', 'sqrt'],
              'min_samples_split': [2, 4]}

grid = GridSearchCV(rf, param_grid, verbose=3)

In [15]:
# Train the model with GridSearch
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END max_features=auto, min_samples_split=2, n_estimators=400;, score=0.903 total time=   6.5s
[CV 2/5] END max_features=auto, min_samples_split=2, n_estimators=400;, score=0.908 total time=   6.2s
[CV 3/5] END max_features=auto, min_samples_split=2, n_estimators=400;, score=0.880 total time=   6.0s
[CV 4/5] END max_features=auto, min_samples_split=2, n_estimators=400;, score=0.883 total time=   4.8s
[CV 5/5] END max_features=auto, min_samples_split=2, n_estimators=400;, score=0.884 total time=   4.6s
[CV 1/5] END max_features=auto, min_samples_split=4, n_estimators=400;, score=0.901 total time=   4.5s
[CV 2/5] END max_features=auto, min_samples_split=4, n_estimators=400;, score=0.902 total time=   4.3s
[CV 3/5] END max_features=auto, min_samples_split=4, n_estimators=400;, score=0.885 total time=   4.3s
[CV 4/5] END max_features=auto, min_samples_split=4, n_estimators=400;, score=0.884 total time=   4.4s
[CV 5/5] END 

GridSearchCV(estimator=RandomForestClassifier(n_estimators=200),
             param_grid={'max_features': ['auto', 'sqrt'],
                         'min_samples_split': [2, 4], 'n_estimators': [400]},
             verbose=3)

In [16]:
print(grid.best_params_)
print(grid.best_score_)

{'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 400}
0.8912804997853281


In [17]:
# Make predictions with hypertuned model
predictions = grid.predict(X_test_scaled)
print(classification_report(encoded_y_test, predictions, 
                            target_names = ["confirmed", "false positive", "candidate"]))

                precision    recall  f1-score   support

     confirmed       0.83      0.77      0.80       411
false positive       0.84      0.85      0.84       484
     candidate       0.97      1.00      0.98       853

      accuracy                           0.90      1748
     macro avg       0.88      0.87      0.87      1748
  weighted avg       0.90      0.90      0.90      1748



# Save the Model

In [18]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'RandomForest.sav'
joblib.dump(rf, filename)

['RandomForest.sav']